In [163]:
import pandas as pd 
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
from keybert import KeyBERT
import matplotlib.pyplot as plt
from IPython.display import display
import time
from tqdm import tqdm
import re
import ast
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager


pd.set_option('display.max_columns', None)

In [7]:


#2021 survey 
stack_2021_a = pd.read_csv("stack_overflow_survey_2021_1.csv")
stack_2021_b = pd.read_csv("stack_overflow_survey_2021_2.csv")

stack_2021 = pd.concat([stack_2021_a, stack_2021_b], ignore_index=True)

cols = ["Country", "EdLevel", "DevType", "LanguageHaveWorkedWith", 
        "DatabaseHaveWorkedWith", "PlatformHaveWorkedWith", "WebframeHaveWorkedWith", 
        "ToolsTechHaveWorkedWith", "MiscTechHaveWorkedWith"]

stack_2021 = stack_2021[cols]

#2023 survey 
stack_2023_a = pd.read_csv("stack_overflow_survey_2023_1.csv")
stack_2023_b = pd.read_csv("stack_overflow_survey_2023_2.csv")

stack_2023 = pd.concat([stack_2023_a, stack_2023_b], ignore_index=True)

cols = ["Country", "EdLevel", "DevType", "LanguageHaveWorkedWith", 
        "DatabaseHaveWorkedWith", "PlatformHaveWorkedWith", "WebframeHaveWorkedWith", 
        "ToolsTechHaveWorkedWith", "MiscTechHaveWorkedWith"]

stack_2023 = stack_2023[cols]

#2024 survey 
stack_2024_a = pd.read_csv("stack_overflow_survey_2024_1.csv")
stack_2024_b = pd.read_csv("stack_overflow_survey_2024_2.csv")

stack_2024 = pd.concat([stack_2024_a, stack_2024_b], ignore_index=True)

cols = ["Country", "EdLevel", "DevType", "LanguageHaveWorkedWith", 
        "DatabaseHaveWorkedWith", "PlatformHaveWorkedWith", "WebframeHaveWorkedWith", 
        "ToolsTechHaveWorkedWith", "MiscTechHaveWorkedWith"]

stack_2024 = stack_2024[cols]

stack_df = pd.concat([stack_2021, stack_2023, stack_2024], ignore_index=True)


In [8]:
stack_df.shape

(152961, 9)

In [9]:
stack_df.head()

,Country,EdLevel,DevType,LanguageHaveWorkedWith,DatabaseHaveWorkedWith,PlatformHaveWorkedWith,WebframeHaveWorkedWith,ToolsTechHaveWorkedWith,MiscTechHaveWorkedWith
0,Slovakia,"Secondary school (e.g. American high school, G...","Developer, mobile",C++;HTML/CSS;JavaScript;Objective-C;PHP;Swift,PostgreSQL;SQLite,NaN,Laravel;Symfony,NaN,NaN
1,Netherlands,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",NaN,JavaScript;Python,PostgreSQL,NaN,Angular;Flask;Vue.js,Docker;Git;Yarn,Cordova
2,Russian Federation,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",NaN,Assembly;C;Python;R;Rust,SQLite,Heroku,Flask,NaN,NumPy;Pandas;TensorFlow;Torch/PyTorch
3,Austria,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Developer, front-end",JavaScript;TypeScript,NaN,NaN,Angular;jQuery,NaN,NaN
4,United Kingdom of Great Britain and Northern I...,"Master’s degree (M.A., M.S., M.Eng., MBA, etc.)","Developer, desktop or enterprise applications;...",Bash/Shell;HTML/CSS;Python;SQL,Elasticsearch;PostgreSQL;Redis,NaN,Flask,Docker;Git;Kubernetes;Yarn,Apache Spark;Hadoop;NumPy;Pandas


Our full stack overflow survey has 152961 rows. 

In [110]:
berkeley = pd.read_csv("UC_Berkeley.csv")
berkeley["University"] = "Berkeley"
berkeley = berkeley[['University', 'Course ID', 'Course Title', 'Upper Div', 'Skills']]

merced = pd.read_csv("UC_Merced.csv")
merced["University"] = "Merced"
merced["Upper Div"] = merced['Upper']
merced = merced[['University', 'Course ID', 'Course Title',  'Upper Div', 'Skills']]

davis = pd.read_csv("UCD.csv")
davis["Skills"] = davis['keywords']
davis["University"] = "Davis"
davis = davis[['University', 'Course ID', 'Course Title',  'Upper Div', 'Skills']]

irvine = pd.read_csv("UCI.csv")
irvine["University"] = "Irvine"
irvine["Upper Div"] = irvine['Upper']
irvine = irvine[['University', 'Course ID', 'Course Title',  'Upper Div', 'Skills']]

la = pd.read_csv("UCLA.csv")
la['University'] = 'LA'
la = la[['University', 'Course ID', 'Course Title', 'Upper Div', 'Skills']]

riverside = pd.read_csv("UCR.csv")
riverside["University"] = "Riverside"
riverside["Upper Div"] = riverside['Upper']
riverside = riverside[['University', 'Course ID', 'Course Title', 'Upper Div', 'Skills']]

sd = pd.read_csv("UCSD.csv")
sd['University'] = 'SD'
sd = sd[['University', 'Course ID', 'Course Title', 'Upper Div', 'Skills']]

sc = pd.read_csv("UCSC.csv")
sc["University"] = "SC"
sc["Upper Div"] = sc['Upper']
sc = sc[['University', 'Course ID', 'Course Title', 'Upper Div', 'Skills']]


sb = pd.read_csv("UCSB.csv")
sb['University'] = 'SB'
sb = sb[['University', 'Course ID', 'Course Title',  'Upper Div', 'Skills']]

uc_df = pd.concat([berkeley, merced, davis, irvine, la, riverside, sd, sc, sb], ignore_index=True)
uc_df


,University,Course ID,Course Title,Upper Div,Skills
0,Berkeley,COMPSCI C8,Foundations of Data Science,False,"['data', 'datasets', 'privacy', 'computational..."
1,Berkeley,COMPSCI 10,The Beauty and Joy of Computing,False,"['programming', 'computing', 'python', 'progra..."
2,Berkeley,COMPSCI W10,The Beauty and Joy of Computing,False,"['programming', 'computing', 'students', 'appl..."
3,Berkeley,COMPSCI 36,CS Scholars Seminar: The Educational Climate i...,False,"['cs36', 'cs61a', 'academia', 'programs', 'cur..."
4,Berkeley,COMPSCI 39,Freshman/Sophomore Seminar,False,"['seminars', 'seminar', 'semester', 'students'..."
...,...,...,...,...,...
581,SB,CMPSC 190AAZZ,Special Topics in Computer Science,True,"['architectures', 'cmpsc', 'programming', 'com..."
582,SB,CMPSC 192,Projects in Computer Science,True,"['gpa', 'undergraduate', 'students', 'projects..."
583,SB,CMPSC 193,Internship in Industry,True,"['students', 'proposal', 'projects', 'supervis..."
584,SB,CMPSC 196,Undergraduate Research,True,"['electives', 'students', 'departmental', 'und..."


In [212]:
def stack_col(col_name):
    stack_col = stack_df[col_name].str.split(';').explode().unique().tolist()
    stack_col = [str(entry).lower() for entry in stack_col if pd.notna(entry)]
    return stack_col

def average_word_occurrences(word, text_list):
    text_list = [str(text).lower() for text in text_list]
    return text_list.count(word)

def check_skills(df, col_name):
    university = df.name
    skills = df['Skills'].dropna().apply(ast.literal_eval)
    skills_list = skills.explode().tolist()

    col_values = stack_col(col_name)  
    counts = [average_word_occurrences(skill, skills_list) for skill in col_values]

    return pd.Series(counts, index=col_values)  



In [213]:

lang_eda = uc_df.groupby('University').apply(check_skills, 'LanguageHaveWorkedWith')
lang_eda.to_csv("lane_eda.csv", index = False)

database_eda = uc_df.groupby('University').apply(check_skills, 'DatabaseHaveWorkedWith') 
database_eda.to_csv("database_eda.csv", index = False)

platform_eda = uc_df.groupby('University').apply(check_skills, 'PlatformHaveWorkedWith')
platform_eda.to_csv("platform_eda.csv", index = False)

webframe_eda = uc_df.groupby('University').apply(check_skills, 'WebframeHaveWorkedWith')
webframe_eda.to_csv("webframe_eda.csv", index = False)

tech_eda = uc_df.groupby('University').apply(check_skills, 'ToolsTechHaveWorkedWith')
tech_eda.to_csv("tech_eda.csv", index = False)

misctech_eda = uc_df.groupby('University').apply(check_skills, 'MiscTechHaveWorkedWith')
misctech_eda.to_csv("misctech_eda.csv", index = False)

C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_12224\2407792409.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lang_eda = uc_df.groupby('University').apply(check_skills, 'LanguageHaveWorkedWith')
C:\Users\Viki Shi\AppData\Local\Temp\ipykernel_12224\2407792409.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  database_eda = uc_df.groupby('University').apply(check_skills, 'DatabaseHaveWorkedWith')
C:\U

In [216]:
eda_final = pd.concat([lang_eda, database_eda, platform_eda, webframe_eda, tech_eda, misctech_eda], axis=1)
eda_final.to_csv("complete_eda.csv")

eda_final

,c++,html/css,javascript,objective-c,php,swift,python,assembly,c,r,rust,typescript,bash/shell,sql,c#,java,node.js,powershell,ruby,perl,matlab,kotlin,julia,haskell,delphi,go,scala,dart,vba,groovy,clojure,apl,lisp,f#,elixir,erlang,crystal,cobol,bash/shell (all shells),ada,lisp,ocaml,raku,zig,solidity,lua,matlab,visual basic (.net),gdscript,cobol,fortran,prolog,apex,sas,nim,flow,micropython,zephyr,postgresql,sqlite,elasticsearch,redis,mysql,microsoft sql server,oracle,mongodb,mariadb,dynamodb,couchbase,firebase,cassandra,ibm db2,supabase,bigquery,cloud firestore,dynamodb,cosmos db,h2,datomic,influxdb,firebase realtime database,microsoft access,firebird,solr,cockroachdb,snowflake,couch db,ravendb,neo4j,clickhouse,duckdb,tidb,presto,databricks sql,eventstoredb,heroku,microsoft azure,aws,google cloud platform,digitalocean,oracle cloud infrastructure,ibm cloud or watson,amazon web services (aws),netlify,vercel,google cloud,openstack,vmware,vultr,cloudflare,firebase,digital ocean,fly.io,render,hetzner,openshift,ovh,managed hosting,oracle cloud infrastructure (oci),"linode, now akamai",colocation,scaleway,ibm cloud or watson,databricks,pythonanywhere,supabase,alibaba cloud,laravel,symfony,angular,flask,vue.js,jquery,express,react.js,angular.js,ruby on rails,django,fastapi,asp.net core,spring,svelte,asp.net,gatsby,drupal,next.js,react,remix,node.js,wordpress,nestjs,angularjs,phoenix,solid.js,asp.net core,elm,nuxt.js,fastify,spring boot,deno,blazor,codeigniter,qwik,play framework,lit,htmx,strapi,yii 2,astro,docker,git,yarn,kubernetes,unity 3d,terraform,flow,unreal engine,ansible,deno,puppet,xamarin,chef,pulumi,npm,pip,vite,webpack,cargo,make,nix,homebrew,pnpm,bun,chocolatey,maven (build tool),msbuild,nuget,visual studio solution,gradle,cmake,gnu gcc,ant,wasmer,boost.test,podman,apt,llvm's clang,composer,bandit,doctest,msvc,dagger,cppunit,google test,pacman,godot,ninja,meson,qmake,catch2,build2,scons,tunit,elfspy,lest,cute,snitch,liblittletest,cordova,numpy,pandas,tensorflow,torch/pytorch,apache spark,hadoop,keras,.net core / .net 5,qt,flutter,.net framework,react native,electron,tauri,rabbitmq,spring framework,scikit-learn,.net (5+),.net framework (1.0 - 4.8),apache kafka,gtk,hugging face transformers,capacitor,mfc,opengl,xamarin,cuda,jax,opencv,ionic,.net maui,swiftui,quarkus,micronaut,ktor,tidyverse,uno platform,ruff,roslyn,directx,opencl,mlflow
University,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Berkeley,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Davis,0,0,0,0,0,0,4,1,0,0,0,0,0,3,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Irvine,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [215]:
corr_sum = pd.DataFrame()
corr_sum["correlation sums"] = eda_final.sum(axis=1)
corr_sum

,correlation sums
University,
Berkeley,3
Davis,10
Irvine,5
LA,3
Merced,7
Riverside,1
SB,9
SC,2
SD,14
